Code of 'DIDL NLP Part' (Partial)

14.3 预训练词嵌入的数据集(p482)

In [6]:
import math
import os
import random
import torch
from d2l import torch as d2l

In [2]:
'''read database'''
d2l.DATA_HUB['ptb'] = (d2l.DATA_URL + 'ptb.zip',
                       '319d85e578af0cdc590547f26231e4e31cdf1e42')

def read_ptb():
    data_dir = d2l.download_extract('ptb')
    with open(os.path.join(data_dir, 'ptb.train.txt')) as f:
        raw_text = f.read()
    return [line.split() for line in raw_text.split('\n')]

sentences = read_ptb()
print(f'# sentence num: {len(sentences)}')

vocab = d2l.Vocab(sentences, min_freq=10)
print(f'vocab size:{len(vocab)}')

# sentence num: 42069
vocab size:6719


In [7]:
'''下采样'''
def subsample(sentences, vocab):
    #exclude '<unk>'
    sentences = [[token for token in line if vocab[token] != vocab.unk] for line in sentences]
    counter = d2l.count_corpus(sentences)
    num_tokens = sum(counter.values())

    #保留词元，true
    def keep(token):
        return (random.uniform(0,1)<math.sqrt(1e-4/counter[token]*num_tokens))
    
    return ([[token for token in line if keep(token)] for line in sentences],counter)

subsampled, counter = subsample(sentences, vocab)

d2l.show_list_len_pair_hist(
    ['origin', 'subsampled'],
    'count', sentences, subsampled
)

AttributeError: module 'd2l.torch' has no attribute 'count_corpus'

14.4 预训练word2vec

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

batch_size, max_window_size, num_noise_word = 512, 5, 5
data_iter, vocab = d2l.load_data_ptb(batch_size, max_window_size, num_noise_word)

In [2]:
'''embedded layer'''
embed = nn.Embedding(num_embeddings=20, embedding_dim=4)
print(f'Parameter embedding_weight({embed.weight.shape})'
      f'dtype={embed.weight.dtype}')

Parameter embedding_weight(torch.Size([20, 4]))dtype=torch.float32


In [3]:
'''forward'''
def skip_gram(center, contexts_and_negatives, embed_v, embed_u):
    v = embed_v(center)
    u = embed_u(contexts_and_negatives)
    pred = torch.bmm(v, u.permute(0,2,1))
    return pred

In [4]:
'''loss function and training'''
class SigmoidBCELoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, inputs, target, mask=None):
        out = nn.functional.binary_cross_entropy_with_logits(
            inputs, target, weight=mask, reduction='none')
        return out.mean(dim=1)
    
loss = SigmoidBCELoss()

embed_size = 100
net = nn.Sequential(nn.Embedding(num_embeddings=len(vocab),
                                 embedding_dim=embed_size),
                    nn.Embedding(num_embeddings=len(vocab),
                                 embedding_dim=embed_size))



In [5]:
def train(net, data_iter, lr, num_epochs, device=d2l.try_gpu()):
    def init_weights(m):
        if type(m) == nn.Embedding:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', xlim=[1, num_epochs])

    #规范化
    metric = d2l.Accumulator(2)
    for epoch in range(num_epochs):
        timer, num_batches = d2l.Timer(), len(data_iter)
        for i, batch in enumerate(data_iter):
            optimizer.zero_grad()
            center, context_negative, mask, label = [
                data.to(device) for data in batch]
            
            pred = skip_gram(center, context_negative, net[0], net[1])
            l = (loss(pred.reshape(label.shape).float(), label.float(), mask)
                    / mask.sum(axis=1) * mask.shape[1])
            l.sum().backward()
            optimizer.step()
            metric.add(l.sum(), l.numel())
            if (i+1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i+1) / num_batches,
                                (metric[0] / metric[1], ))
    
    print(f'loss{metric[0]/metric[1]:.3f},'
            f'{metric[1] / timer.stop():.1f} tokens/sec on {str(device)}')

In [6]:
lr, num_epochs = 0.002, 2
train(net, data_iter, lr, num_epochs)

AttributeError: Can't pickle local object 'load_data_ptb.<locals>.PTBDataset'

: 

In [ ]:
'''应用词嵌入，寻找语义相近词'''
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[vocab[query_token]]
    cos = torch.mv(W, x) / torch.sqrt(torch.sum(W*W, dim=1)*
                                      torch.sum(x*x) + 1e-9)
    topk = torch.topk(cos, k=k+1)[1].cpu().numpy().astype('int32')
    for i in topk[1:]: #delete input
        print(f'cosine sim = {float(cos[i]):.3f}: {vocab.to_tokens(i)}')

get_similar_tokens('chip', 3, net[0])

cosine sim = 0.332: testify
cosine sim = 0.307: tickets
cosine sim = 0.297: tape
